## Build the feature matrix 
##### Features: 2kb windows
##### Cells: Aggregated ATAC nuclei

In [59]:
import os
import csv
import scipy
import scanpy as sc
import anndata as ad
import numpy as np
import pandas as pd
import episcanpy.api as epi
%config InlineBackend.figure_format = "retina"
os.chdir("/data/proj/GCB_MK/CZI/episcanpy_analysis/AGG_ATAC_210218/")

In [9]:
RESULTS_FILE = "data/Binned_2kb_count_AGG_ATAC_results.h5ad"
BINS_PATH = "2kb_binned_hg38.bed"
FRAG_PATH = "outs/fragments.tsv.gz"
CSV_PATH = "outs/singlecell_EPI.csv"
METADATA_PATH = "sample_randomizer_metadata.csv"
GEM_TO_NGI_ID = {"1" : "P20056_1001",
                "2" : "P20056_1002",
                "3" : "P20056_1003",
                "4" : "P20056_1004", 
                "5" : "P20057_1001",
                "6" : "P20057_1002",
                "7" : "P20057_1003"}
NGI_ID_TO_GEM = {v: k for k, v in GEM_TO_NGI_ID.items()}

In [10]:
annot = epi.ct.load_features(BINS_PATH)

In [11]:
## build the matrix (takes a LOT of time)
counts = epi.ct.bld_mtx_fly(tsv_file = FRAG_PATH,
                            csv_file = CSV_PATH,
                           annotation = annot,
                           save = "data/binned_2kb_count_matrix.h5ad")

loading barcodes
building count matrix


100%|██████████| 1544146/1544146 [2:16:19<00:00, 188.79it/s]  


building AnnData object
filtering barcodes


In [12]:
adata = ad.read("data/binned_2kb_count_matrix.h5ad")
adata

AnnData object with n_obs × n_vars = 19062 × 1544146

In [14]:
bcd_list = adata.obs_names.to_list()
NGI_ID_list = [GEM_TO_NGI_ID[bcd.split("-")[-1]] for bcd in bcd_list] 
adata.obs["NGI_ID"] = NGI_ID_list

meta = pd.read_csv(METADATA_PATH, sep=",")
meta = meta[meta["NGI_ID"].notna()]

sample_attributes = meta.columns.to_list()
sample_attributes.remove("NGI_ID") #already added, not needed again

#populates anndata.obs
for attribute in sample_attributes:
    adata.obs[attribute] = [meta.loc[meta.NGI_ID == i, attribute].iloc[0] for i in NGI_ID_list]

In [87]:
## get barcodes of doublets identified from ArchR.
ARCHR_ALL_BCD = "./archr_all_bcd.csv"
ARCHR_SINGLET_BCD = "./archr_singlet_bcd.csv"

with open(ARCHR_ALL_BCD, newline='') as f:
    reader = csv.reader(f)
    archr_all = list(reader)

with open(ARCHR_SINGLET_BCD, newline='') as f:
    reader = csv.reader(f)
    archr_singlets = list(reader)
    
#change archr_barcode naming style to match anndata.obs
archr_list = [archr_all, archr_singlets]
idx = 0
for ls in archr_list:
    tmp = []
    for archr_bcd in ls[1:]:
        ngi_id,bcd = archr_bcd[-1].split("#")
        tmp.append(bcd[:-1] + NGI_ID_TO_GEM[ngi_id])
    idx += 1
    if idx == 1:
        archr_all = tmp
    else:
        archr_singlets = tmp

# get doublet barcodes
archr_doublets = [bcd for bcd in archr_all if bcd not in archr_singlets]
#add doublet attribute
adata.obs["ArchR_doublet"] = ["True" if bcd in archr_doublets  else "False" for bcd in bcd_list]

In [22]:
# add gene annotations 
epi.tl.find_genes(adata,
           gtf_file='ref/gencode.v35.annotation.gtf',
           key_added='transcript_annotation',
           upstream=2000,
           feature_type='transcript',
           annotation='HAVANA',
           raw=False)

In [ ]:
# get "commonness" of each feature (make sure matrix is binarized)
n_cells = np.sum(adata.X, axis=0).tolist()
adata.var["n_cells"] = n_cells[0]

#### Saving the 2kb count matrix before filtering top 60,000 features

In [123]:
epi.pp.binarize(adata)
adata.write("data/populated_binned_2kb_count_matrix.h5ad")

In [124]:
adata

AnnData object with n_obs × n_vars = 19062 × 1544146
    obs: 'NGI_ID', 'ProcessNumber', 'caseNO', 'Tissue', 'Sex', 'Age', 'PMI', 'MK_ID', '10X_BATCH', 'NGS_BATCH', 'ArchR_doublet'
    var: 'transcript_annotation', 'n_cells'

### subset anndata by selecting only the top 20,000 features 

In [125]:
small_adata = adata[:, adata.var["n_cells"] >= np.sort(adata.var["n_cells"])[-20000]]
small_adata

In [127]:
#save 
small_adata.write("data/top60k_binned_2kb_count_matrix.h5ad")